In [27]:
# Import TensorFlow and TensorFlow Datasets

import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import warnings
warnings.filterwarnings("ignore")

import os

Load the MNIST dataset and split it into training and test chunks.

In [28]:
# Load the dataset we'll use for this lab
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True, data_dir='./data')

mnist_train, mnist_test = datasets['train'], datasets['test']

# Mirrored strategy on 4 GPUs

In [29]:
# Define the strategy to use and print the number of devices found
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


Number of devices: 4


In [30]:
# Get the number of examples in the train and test sets
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
# Use for Mirrored Strategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
# Use for No Strategy
#BATCH_SIZE = BATCH_SIZE_PER_REPLICA * 1

A mapping function which normalizes your images:

In [31]:
# Function for normalizing the image
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

In [32]:
# Set up the train and eval data set
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

For your model to follow the strategy, define your model within the strategy's scope.
- Run all the cells below and notice the results. 
- Afterwards comment out `with strategy.scope():` and run everything again, without the strategy. 
Then you can compare the results. 
The important thing to notice and compare is the time taken for each epoch to complete. As mentioned in the lecture, doing a mirrored strategy on a single device (which our lab environment has) might take longer to train because of the overhead in implementing the strategy. With that, the advantages of using this strategy is more evident if you will use it on multiple devices.

In [33]:
# Use for Mirrored Strategy -- comment out `with strategy.scope():` and deindent for no strategy
with strategy.scope():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
    ])

In [34]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [35]:
model.fit(train_dataset, epochs=12)

2022-06-26 10:31:46.949049: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:454] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.


Epoch 1/12
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1


235/235 [==============================] - 6s 5ms/step - loss: 0.6597 - accuracy: 0.8277
Epoch 2/12
235/235 [==============================] - 1s 3ms/step - loss: 0.1138 - accuracy: 0.9676
Epoch 3/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0688 - accuracy: 0.9801
Epoch 4/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0549 - accuracy: 0.9850
Epoch 5/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0432 - accuracy: 0.9877
Epoch 6/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0357 - accuracy: 0.9893
Epoch 7/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0321 - accuracy: 0.9904
Epoch 8/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0270 - accuracy: 0.9921
Epoch 9/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0233 - accuracy: 0.9929
Epoch 10/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0203 - accuracy: 0.9945
Epoch 11/1

# Without mirrored strategy

In [36]:
# Get the number of examples in the train and test sets
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
# Use for Mirrored Strategy
# BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
# Use for No Strategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * 1

In [37]:
model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
    ])

In [38]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [39]:
model.fit(train_dataset, epochs=12)

Epoch 1/12
235/235 [==============================] - 1s 4ms/step - loss: 0.6439 - accuracy: 0.8281
Epoch 2/12
235/235 [==============================] - 1s 3ms/step - loss: 0.1140 - accuracy: 0.9666
Epoch 3/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0743 - accuracy: 0.9780
Epoch 4/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0580 - accuracy: 0.9831
Epoch 5/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.9867
Epoch 6/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0411 - accuracy: 0.9875
Epoch 7/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0326 - accuracy: 0.9898
Epoch 8/12
235/235 [==============================] - 1s 2ms/step - loss: 0.0272 - accuracy: 0.9925
Epoch 9/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0231 - accuracy: 0.9935
Epoch 10/12
235/235 [==============================] - 1s 3ms/step - loss: 0.0202 - accuracy: 0.9940

In [40]:
!nvidia-smi

Sun Jun 26 10:32:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   42C    P0    66W / 300W |  16002MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   

In [41]:
!hostname

cg32-3.agave.rc.asu.edu
